In [1]:
!pip install -r requirements.txt

In [ ]:
!pip install --upgrade transformers datasets evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement BitsAndBytesConfig (from versions: none)
ERROR: No matching distribution found for BitsAndBytesConfig


In [1]:
import transformers
print(transformers.__version__)


4.46.3


In [1]:
import torch
import os
import transformers
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

# 1. 설정: pandas 출력 옵션 및 시드 고정
pd.set_option('display.max_columns', None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)

In [3]:
# 2. 모델 및 토크나이저 로드 (8-bit 양자화)
model_name = "sh2orc/Llama-3.1-Korean-8B-Instruct"

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,            # 8-bit 정밀도로 모델 로드
#     llm_int8_threshold=6.0,       # int8 양자화 임계값
#     llm_int8_has_fp16_weight=False # FP16 가중치 사용 여부
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. PEFT 설정 (LoRA)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
dataset_path = '/data/data/train.csv'  # Update this path as needed
dataset = pd.read_csv(dataset_path) 

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        

df = pd.DataFrame(records)

In [14]:
answer_num = df.iloc[1]["answer"]
choices = df.iloc[1]["choices"]

# if isinstance(answer_num, int) and 1 <= answer_num <= 5:
#         output = choices[answer_num - 1]  # 정답 텍스트
# else:
#     output = "정답 없음" 
    
output = choices[answer_num - 1]
print(output)

고려 문종 때에 남경(南京)으로 승격되었다.


In [15]:
# 프롬프트 템플릿 정의
PROMPT_TEMPLATE = """다음 지문을 읽고 질문에 답하세요.

지문:
{paragraph}

질문:
{question}

{question_plus}
선택지:
{choices}

정답을 선택지 중에서 선택하여 작성하세요:
"""


In [16]:
processed_dataset = []
for i in range(len(df)):
    choices = df.iloc[i]["choices"]
    choices_string = "\n".join([f"{idx + 1}. {choice}" for idx, choice in enumerate(choices)])
    paragraph = df.iloc[i]["paragraph"]
    question = df.iloc[i]["question"]
    answer_num = df.iloc[i]["answer"]
    question_plus = df.iloc[i].get("question_plus", None)

    # 정답 번호에 대응하는 텍스트
    # if isinstance(answer_num, int) and 1 <= answer_num <= len(choices):
    #     output = choices[answer_num - 1]  # 정답 텍스트
    # else:
    #     output = "정답 없음"  # 정답이 없거나 유효하지 않을 경우

    output = choices[answer_num - 1]
    
    # 입력 데이터 생성
    input_text = PROMPT_TEMPLATE.format(
        paragraph=paragraph,
        question=question,
        question_plus=f"<보기>:\n{question_plus}" if question_plus else "",  # 'question_plus' 처리
        choices=choices_string,
    )

    processed_dataset.append(
        {
            "input": input_text,
            "output": output,
        }
    )

processed_df = pd.DataFrame(processed_dataset)
processed_dataset = Dataset.from_pandas(processed_df)



In [17]:
#데이터의 최대 길이 계산
max_num = 0

for _ in range(len(processed_dataset)):
    data_length = len(processed_dataset[i]["input"] + processed_dataset[i]["output"])
    if data_length > max_num:
        max_num = data_length
        
print(max_num)




797


In [18]:
def tokenize_function(examples):
    inputs = examples["input"]
    outputs = examples["output"]

   
    full_texts = [inp + tokenizer.eos_token + out for inp, out in zip(inputs, outputs)]

    
    tokenized_inputs = tokenizer(
        full_texts,
        max_length=1024,
        truncation=True,
    )

   
    labels = tokenized_inputs["input_ids"].copy()

    
    for i in range(len(inputs)):
        input_ids = tokenizer(
            inputs[i],
            max_length=1024,
            truncation=True,
        )["input_ids"]
        input_len = len(input_ids)
        
        
        labels[i] = torch.tensor(labels[i]) 
        labels[i][:input_len] = -100 
        labels[i] = labels[i].tolist()  

    tokenized_inputs["labels"] = labels

    return tokenized_inputs


tokenized_datasets = processed_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=processed_dataset.column_names,
)

Map:   0%|          | 0/2031 [00:00<?, ? examples/s]

In [19]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1, seed=42)
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['test']

# 7. 데이터 콜레이터 정의
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [20]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=decoded_preds, references=decoded_labels)


In [21]:
training_args = TrainingArguments(
    output_dir="./outputs_llama",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="steps",
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    fp16=True,
    dataloader_num_workers=4,
    report_to="none",
)

# 10. Trainer 초기화 및 훈련
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_661913/3139246749.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
# 15. 훈련 시작
trainer.train()
trainer.save_model("./fine_tuned_llama8B_model")

Step,Training Loss,Validation Loss
500,1.955400,1.876130
1000,1.897700,1.863594


### 추론단계

In [5]:
!pip install rank-bm25

  Obtaining dependency information for rank-bm25 from https://files.pythonhosted.org/packages/2a/21/f691fb2613100a62b3fa91e9988c991e9ca5b89ea31c0d3152a3210344f9/rank_bm25-0.2.2-py3-none-any.whl.metadata


In [1]:
import torch
import os
import transformers
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
from torch.utils.data import DataLoader
import json
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling,DataCollatorWithPadding
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss
from rank_bm25 import BM25Okapi
import random


# 1. 설정: pandas 출력 옵션 및 시드 고정
pd.set_option('display.max_columns', None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)

In [3]:
from peft import PeftModel

# 2. 모델 및 토크나이저 로드
model_name = "./fine_tuned_llama8B_model"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# 3. PEFT 설정 (LoRA)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델을 평가 모드로 설정
model.eval()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# 데이터 로드
dataset_path = '/data/data/test.csv'  # 필요한 경로로 업데이트하세요
dataset = pd.read_csv(dataset_path)

# JSON 형태로 저장된 'problems' 컬럼을 펼치기
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        "question_plus": problems.get('question_plus', None),
    }
    records.append(record)

df = pd.DataFrame(records)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# 프롬프트 템플릿 정의
PROMPT_TEMPLATE = """다음 지문을 읽고 질문에 답하세요.

지문:
{paragraph}

질문:
{question}

{question_plus}
선택지:
{choices}

정답을 선택지 중에서 선택하여 작성하세요:
"""

In [5]:
processed_dataset = []
for i in range(len(df)):
    choices = df.iloc[i]["choices"]
    choices_string = "\n".join([f"{idx + 1}. {choice}" for idx, choice in enumerate(choices)])
    paragraph = df.iloc[i]["paragraph"]
    question = df.iloc[i]["question"]
    question_plus = df.iloc[i].get("question_plus", None)

    # 입력 데이터 생성
    input_text = PROMPT_TEMPLATE.format(
        paragraph=paragraph,
        question=question,
        question_plus=f"<보기>:\n{question_plus}" if question_plus else "",
        choices=choices_string,
    )

    processed_dataset.append(
        {
            "id": df.iloc[i]["id"],
            "input": input_text,
            "choices": choices,
        }
    )

processed_df = pd.DataFrame(processed_dataset)

In [6]:
# 배치 단위 설정
batch_size = 1  # 원하는 배치 크기로 설정

# 텍스트 생성 함수 정의 (배치 처리)
def generate_answers(input_texts):
    inputs = tokenizer(
        input_texts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=1024,
    ).to(device)

    with torch.no_grad():
        # 답변 생성
        outputs = model.generate(
            **inputs,
            max_length=1024 + 512,  # 입력 길이 + 예상 답변 길이
            num_beams=5,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2,
        )

    # 생성된 텍스트 디코딩
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # 입력 텍스트 부분을 제외하고 답변만 추출
    generated_answers = []
    for input_text, generated_text in zip(input_texts, generated_texts):
        answer = generated_text[len(input_text):].strip()
        generated_answers.append(answer)

    return generated_answers

In [7]:
# 유사도 계산 및 가장 유사한 선택지 선택 함수 정의
def select_best_choices(generated_answers, choices_list):
    selected_indices = []
    for generated_answer, choices in zip(generated_answers, choices_list):
        # 선택지와 생성된 답변을 토큰화
        tokenized_choices = [tokenizer.tokenize(choice) for choice in choices]
        tokenized_answer = tokenizer.tokenize(generated_answer)

        # BM25 객체 생성
        bm25 = BM25Okapi(tokenized_choices)

        # 유사도 스코어 계산
        scores = bm25.get_scores(tokenized_answer)

        # 가장 높은 스코어의 선택지 선택
        best_choice_index = int(np.argmax(scores))
        selected_indices.append(best_choice_index + 1)  # 선택지는 1부터 시작하므로 +1

    return selected_indices


### 전체 추론 후 저장

In [8]:
results = []

# 배치 단위로 데이터 처리
for idx in tqdm(range(0, len(processed_df), batch_size)):
    batch = processed_df.iloc[idx:idx+batch_size]
    input_texts = batch['input'].tolist()
    choices_list = batch['choices'].tolist()
    ids = batch['id'].tolist()

    # 모델을 사용하여 답변 생성
    generated_answers = generate_answers(input_texts)

    # 가장 유사한 선택지 선택
    answer_indices = select_best_choices(generated_answers, choices_list)

    # 결과 저장
    for id_, answer_index in zip(ids, answer_indices):
        results.append({
            'id': id_,
            'answer': answer_index,
        })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# CSV로 저장
output_path = 'output.csv'  # 필요한 경로로 업데이트하세요
results_df.to_csv(output_path, index=False)


  0%|          | 0/109 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  1%|          | 1/109 [01:35<2:52:01, 95.57s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  3%|▎         | 3/109 [04:47<2:49:43, 96.07s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  4%|▎         | 4/109 [06:24<2:48:39, 96.37s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-o

### 한행만 추론

In [8]:
# 테스트 데이터셋: 첫 번째 행만 선택
test_batch = processed_df.iloc[:1]  # 첫 번째 행 선택
input_texts = test_batch['input'].tolist()
choices_list = test_batch['choices'].tolist()
ids = test_batch['id'].tolist()

# 모델을 사용하여 답변 생성
generated_answers = generate_answers(input_texts)

# 가장 유사한 선택지 선택
answer_indices = select_best_choices(generated_answers, choices_list)

# 결과 저장
results = []
for id_, answer_index in zip(ids, answer_indices):
    results.append({
        'id': id_,
        'answer': answer_index,
    })

# 결과 출력
results_df = pd.DataFrame(results)
print(results_df)
print(generated_answers)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


                     id  answer
0  generation-for-nlp-0       4
["다시 한 번 읽어보세요. 윗 글에서 가장 중요하게 강조한 내용은 '독자'와 '책'의 관계에 관한 것이었습니다. 따라서, 이와 관련이 없는 선택지를 찾아야 합니다. 선택지는 총 다섯 가지로 구성되어 있습니다. 첫 번째는 '같은 책'에 대한 내용입니다. 두 번째와 세 번째, 그리고 네 번째가 모두 독자를 중심으로 한 내용이기 때문에, 가장 적합한 선택지가 될 가능성이 높습니다. 다만, '읽기' 환경과 '과제'가 '배경' 지식을 '관점'보다 덜 중요하다고 할 수 있지만, 여전히 독자기능에 큰 영향을 미치는 요소들로 간주됩니다. 마지막으로, 책 속의 '사회'나 '시대'과 필자의 '대화'를 중점적으로 다루고 있으므로 '5'라는 번호를 선택하는 것이 가장 합리적인 선택일 것입니다."]
